In [1]:
# pip install geotext
# from geotext import GeoText
# import requests
# from bs4 import BeautifulSoup
import lxml.html
from selenium import webdriver
import stanza
nlp = stanza.Pipeline('en',processors='tokenize,mwt,pos,lemma,depparse', verbose = True);

2022-03-07 16:18:13 WARNING: Can not find mwt: default from official model list. Ignoring it.
2022-03-07 16:18:13 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |
| depparse  | combined |

2022-03-07 16:18:13 INFO: Use device: cpu
2022-03-07 16:18:13 INFO: Loading: tokenize
2022-03-07 16:18:13 INFO: Loading: pos
2022-03-07 16:18:15 INFO: Loading: lemma
2022-03-07 16:18:15 INFO: Loading: depparse
2022-03-07 16:18:16 INFO: Done loading processors!


# Weather forecasting

In [2]:
def weather_forecast(query):
    postagSent= nlp(query)
    city = ''
    for i in postagSent.to_dict()[0]:
        if i['xpos']=='NNP':
            city = i['lemma']
    # creating url and requests instance
    url = 'https://www.google.com/search?q='+city+ '+weather'

    options = webdriver.ChromeOptions()
    prefs = {
      "translate_whitelists": {"your native language":"en"},
      "translate":{"enabled":"True"}
    }
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(executable_path='H://Downloads//chromedriver_win32 (2)//chromedriver.exe',options=options)
    driver.get(url)

    tree = lxml.html.fromstring(driver.page_source)
    # Now you can query:
    driver.close()
    
    
    daysofweek = {'måndag':'Monday',
                'tisdag':'Tuesday',
                'onsdag':'Wednesday',
                'torsdag':'Thursday',
                'fredag':'Friday',
                'lördag' :'Saturday',
                'söndag':'Sunday'}
    
    # With your xpath there are 2 results...
    currentTemperature = tree.xpath('//*[@id="wob_tm"]/text()')[0]+'°C'
    today = daysofweek[tree.xpath('//*[@id="wob_dts"]/text()')[0].split()[0]]
    time = tree.xpath('//*[@id="wob_dts"]/text()')[0].split()[1]
    maxTempTod = tree.xpath('//*[@id="wob_dp"]/div[1]/div[3]/div[1]/span[1]/text()')
    minTempTod = tree.xpath('//*[@id="wob_dp"]/div[1]/div[3]/div[2]/span[1]/text()')
    precipitation = tree.xpath('//*[@id="wob_pp"]/text()')
    humidity = tree.xpath('//*[@id="wob_hm"]/text()')
    windSpeed = tree.xpath('//*[@id="wob_ws"]/text()')
    days=[ 'Monday', 'Tuesday','Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    adapteddays=days[days.index(today):]+days[:days.index(today)]
    
    
    
    minmaxtemps = []
    for i in range(len(days)):
        minmaxtemps.append((tree.xpath('//*[@id="wob_dp"]/div['+str(i+1)+']/div[3]/div[1]/span[1]/text()')[0],tree.xpath('//*[@id="wob_dp"]/div['+str(i+1)+']/div[3]/div[2]/span[1]/text()')[0]))


    # printing all data
    print('City: '+city[0].upper()+city[1:].lower())
    print("Today: "+ today)
    print("Time: "+ time)
    print("Current temperature: "+ currentTemperature)
    print("Precipitation: "+ precipitation[0])
    print("Humidity: "+ humidity[0])
    print("windSpeed: "+ windSpeed[0])
    print('Minimum and maximum temperatures:')
#     print("Temperature today: "+'('+minmaxtemps[0][0]+'°C,'+minTempTod[0][1]+'°C)')
    for i in range(len(days)):
        print(adapteddays[i]+": "+'('+minmaxtemps[i][0]+'°C,'+minmaxtemps[i][1]+'°C)')

In [3]:
weather_forecast('I want to know the weather in stockholm.')

City: Stockholm
Today: Monday
Time: 16:00
Current temperature: 6°C
Precipitation: 2%
Humidity: 58%
windSpeed: 3 m/s
Minimum and maximum temperatures:
Monday: (7°C,-3°C)
Tuesday: (7°C,-3°C)
Wednesday: (6°C,-1°C)
Thursday: (4°C,-1°C)
Friday: (6°C,-2°C)
Saturday: (7°C,-2°C)
Sunday: (7°C,-3°C)


# Finding a restaurant near current location

In [10]:
def restaurant():
    # creating url and requests instance
    url = 'https://www.google.com/search?q=restaurants'

    options = webdriver.ChromeOptions()
    prefs = {
      "translate_whitelists": {"your native language":"en"},
      "translate":{"enabled":"True"}
    }
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(executable_path='H://Downloads//chromedriver_win32 (2)//chromedriver.exe',options=options)
    driver.get(url)
    tree = lxml.html.fromstring(driver.page_source)
    driver.close()    

    restaurantName = tree.xpath('//*[@id="tsuid38"]/div/div/a/div/div[2]/div[1]/span/text()')[0]

    #rating = tree.xpath('//*[@id="_4_ElYriQL8GExc8P6YeamAg43"]/div[1]/div[2]/div[2]/div/div/span[1]/text()')[0]
    #Foodtype = tree.xpath('//*[@id="_4_ElYriQL8GExc8P6YeamAg43"]/div[1]/div[2]/div[3]/div/span[2]/text()')[0]

    restaurantqueryname = ''
    for i in range(len(restaurantName.split())):
        if i == len(restaurantName.split())-1:
            restaurantqueryname+=restaurantName.split()[i]
        else:
            restaurantqueryname+=(restaurantName.split()[i]+'+')


    url2 = 'https://www.google.com/search?q='+restaurantqueryname

    options = webdriver.ChromeOptions()
    prefs = {
      "translate_whitelists": {"your native language":"en"},
      "translate":{"enabled":"True"}
    }
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(executable_path='H://Downloads//chromedriver_win32 (2)//chromedriver.exe',options=options)
    driver.get(url2)

    tree = lxml.html.fromstring(driver.page_source)

    driver.close()

    address = tree.xpath('//*[@id="kp-wp-tab-overview"]/div[1]/div/div/div/div/div/div[5]/div/div/div/span[2]/text()')[0]

    telephone = tree.xpath('//*[@id="kp-wp-tab-overview"]/div[1]/div/div/div/div/div/div[8]/div/div/div/span[2]/span/a/span/span/text()')[0]

    if restaurantName != '':
        print('Restaurant name: '+restaurantName)
        print('Address: '+address)
        print('Telephone: '+telephone)
    else:
        print('No restaurants found in the current area :(')

In [11]:
restaurant()

Restaurant name: Smokey Dog
Address: Mölndalsvägen 87a, 412 63 Göteborg
Telephone: 072-165 05 58


# Finding next tram/bus from current location to a given destination

In [12]:
def nextTram(location):
    url = 'https://www.google.com/search?q=next+tram+going+to+'+location

    options = webdriver.ChromeOptions()
    prefs = {
      "translate_whitelists": {"your native language":"en"},
      "translate":{"enabled":"True"}
    }
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(executable_path='H://Downloads//chromedriver_win32 (2)//chromedriver.exe',options=options)
    driver.get(url)
    tree = lxml.html.fromstring(driver.page_source)
    driver.close()    


    tramnumber = ''
    direction = ''
    if len(tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[1]/div[2]/div[2]/span[1]/text()'))>0:
        if tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[1]/div[2]/div[2]/span[1]/text()')[0].isnumeric() == True:
            tramnumber = tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[1]/div[2]/div[2]/span[1]/text()')[0]



            direction = ''
            if len(tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[1]/div[2]/div[2]/span[2]/text()'))>0:
                direction = tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[1]/div[2]/div[2]/span[2]/text()')[0]

            fromStation=''
            if len(tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[1]/div[1]/span[3]/text()')) >0:    
                fromStation = tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[1]/div[1]/span[3]/text()')[0]

            toStation=''
            if len(tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[1]/div[3]/span[3]/text()'))>0:
                toStation = tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[1]/div[3]/span[3]/text()')[0]

            tramTime=''
            if len(tree.xpath('//*[@id="exp0"]/div[1]/div/div/div[2]/text()'))>0:
                tramTime = tree.xpath('//*[@id="exp0"]/div[1]/div/div/div[2]/text()')[0][:5]

            expectedReachTime=''
            if len(tree.xpath('//*[@id="exp0"]/div[1]/div/div/div[2]/text()'))>0:
                expectedReachTime = tree.xpath('//*[@id="exp0"]/div[1]/div/div/div[2]/text()')[0][-5:]

            tripduration=''
            if len(tree.xpath('//*[@id="exp0"]/div[1]/div/div/div[1]/text()'))>0:
                tripduration = tree.xpath('//*[@id="exp0"]/div[1]/div/div/div[1]/text()')[0]


            print('The next tram/bus is at: '+ tramTime)
            print('Tram/Bus number: '+ tramnumber)
            print('Towards direction: '+ direction)
            print('Departure station: '+ fromStation)
            print('Arrival station: '+ toStation)
            print('Estimated arrival time: '+expectedReachTime)
            print('Estimated trip duration: '+ tripduration)


        elif tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[2]/div[2]/div[2]/span[1]/text()')[0].isnumeric() == True:
            tramnumber = tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[2]/div[2]/div[2]/span[1]/text()')[0]

            direction = ''
            if len(tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[2]/div[2]/div[2]/span[2]/text()'))>0:
                direction = tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[2]/div[2]/div[2]/span[2]/text()')[0]

            fromStation=''
            if len(tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[2]/div[1]/span[3]/text()')) >0:    
                fromStation = tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[2]/div[1]/span[3]/text()')[0]

            toStation=''
            if len(tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[2]/div[3]/span[3]/text()'))>0:
                toStation = tree.xpath('//*[@id="exp0"]/div[2]/div/ol/li[2]/div[3]/span[3]/text()')[0]

            tramTime=''
            if len(tree.xpath('//*[@id="exp0"]/div[1]/div/div/div[2]/text()'))>0:
                tramTime = tree.xpath('//*[@id="exp0"]/div[1]/div/div/div[2]/text()')[0][:5]

            expectedReachTime=''
            if len(tree.xpath('//*[@id="exp0"]/div[1]/div/div/div[2]/text()'))>0:
                expectedReachTime = tree.xpath('//*[@id="exp0"]/div[1]/div/div/div[2]/text()')[0][-5:]

            tripduration=''
            if len(tree.xpath('//*[@id="exp0"]/div[1]/div/div/div[1]/text()'))>0:
                tripduration = tree.xpath('//*[@id="exp0"]/div[1]/div/div/div[1]/text()')[0]

            print('The next tram/bus is at: '+ tramTime)
            print('Tram/Bus number: '+ tramnumber)
            print('Towards direction: '+ direction)
            print('Departure station: '+ fromStation)
            print('Arrival station: '+ toStation)
            print('Estimated arrival time: '+expectedReachTime)
            print('Estimated trip duration: '+ tripduration)

        else:
            print('Transport data not available for this location :(')
    else:
        print('Transport data not available for this location :(')

In [13]:
nextTram('Haga')

The next tram/bus is at: 16:22
Tram/Bus number: 2
Towards direction: mot Göteborg Axel Dahlströms torg
Departure station: Göteborg Varbergsgatan
Arrival station: Handelshögskolan
Estimated arrival time: 16:52
Estimated trip duration: 30 min


# Dialogue system

"""
structure:

user greets

AI greets

user asks

AI grounds if the question is understandable
else: asks the user to repeat the question in another format given examples
(Do you want to know the weather forecast? 
OR would you like to know the transport info ro a given place?
OR you want to know a restaurants name and contact info near you?)

User asks again

AI understands, grounds, and display info else repeat the last AI step

user grounds and greets

AI greets and close

"""

In [ ]:
def getinfo(context,query):
    # displaying answer for the requested question 
    if context == 'restaurant':
        print('AI: Displaying results for a restaurant near by.')
        print('\n')
        restaurant()
        print('\n')
        
    elif context == 'weather':
        postagSent= nlp(query)
        city = ''
        for i in postagSent.to_dict()[0]:
            if i['xpos']=='NNP':
                city = i['lemma']
        print('AI: Displaying results for a the weather\'s forescast in '+city+'.')
        print('\n')
        weather_forecast(city)
        print('\n')
        
    elif context == 'transport':
        postagSent= nlp(query)
        location = ''
        for i in postagSent.to_dict()[0]:
            if i['xpos']=='NNP':
                location = i['lemma']
        print('\n')
        nextTram(location)
        print('\n')
    
    return True

In [124]:
def get_context(query,restaurantkeywords,transportkeywords,weatherkeywords):
    context = ''
    while context =='':
        if len(set(query.split()).intersection(set(restaurantkeywords)))> 0:
            context = 'restaurant'
            print('\n')
            print("AI: Okay! I understood that you are searching for a restaurant.")
            print('\n')
            break    

        elif len(set(query.split()).intersection(set(transportkeywords)))> 0:
            context = 'transport'
            postagSent= nlp(query)
            location = ''
            for i in postagSent.to_dict()[0]:
                if i['xpos']=='NNP':
                    location = i['lemma']
            print('\n')
            print("AI: Okay! I understood that you are searching for the next tram time.")
            print('\n')
            break

        elif len(set(query.split()).intersection(set(weatherkeywords)))> 0:
            context = 'weather'
            postagSent= nlp(query)
            city = ''
            for i in postagSent.to_dict()[0]:
                if i['xpos']=='NNP':
                    city = i['lemma']
            print('\n')
            print("AI: Okay! I understood that you are searching for the weather's forecast in "+ city + ".")
            print('\n')
            break
    return context

In [137]:
def ask_for_more_requests(restaurantkeywords,transportkeywords,weatherkeywords,yesKeywords,noKeywords,inputbool):
    
    print('AI: Do you have any other requests?')
    print('\n')

    inputbool = input('User:')

    if len(set(inputbool.split()).intersection(set(noKeywords))) < 1 and get_context(inputbool,restaurantkeywords,transportkeywords,weatherkeywords) in ['restaurant','transport','weather']:
        context = get_context(inputbool,restaurantkeywords,transportkeywords,weatherkeywords)
        getinfo(context,inputbool)
    return inputbool

In [138]:
def dialogue():
    print('AI: Hello! How can I help?')
    print('\n')
    
    query = input('User: ')
    
    #using set, so as list of keywords gets bigger, searching speed is fast
    restaurantkeywords = set(['restaurant','food','eat','drink','meal','meals','restaurants'])
    transportkeywords = set(['bus','buses','tram','trams','stop','stops','station','stations'])
    weatherkeywords =   set(['weather','forecast','forecasts','temperature','sunny',
                         'cloudy','windy','cloud','clouds','humidity','humid'])
    
    
    #get context of the sentence

    context = get_context(query,restaurantkeywords,transportkeywords,weatherkeywords)
    
    #print the requested information
    getinfo(context,query)
            
    #ask for other requests 
    yesKeywords = set(['yes','Yes','please','need','want','find'])
    noKeywords = set(['no','nope','No','thanks'])
    print('\n')
    inputbool = ''
    while len(set(inputbool.split()).intersection(set(noKeywords))) < 1:
        inputbool = ask_for_more_requests(restaurantkeywords,transportkeywords,weatherkeywords,yesKeywords,noKeywords,inputbool)

    print('\n')
    print('AI: Have a good day, bye! :)')
    return None

In [139]:
dialogue()

AI: Hello! How can I help?


User: Hello! I want to find a restaurant near by please!


AI: Okay! I understood that you are searching for a restaurant.


AI: Displaying results for a restaurant near by.


Restaurant name: Almedal restaurang och pizzeria
Address: Ebbe Lieberathsgatan 18C, 412 65 Göteborg
Telephone: 031-40 99 06




AI: Do you have any other requests?


User:Yes, I want to know the next tram time to Haga


AI: Okay! I understood that you are searching for the next tram time.




AI: Okay! I understood that you are searching for the next tram time.




The next tram/bus is at: 19:42
Tram/Bus number: 2
Towards direction: mot Göteborg Axel Dahlströms torg
Departure station: Göteborg Varbergsgatan
Arrival station: Handelshögskolan
Estimated arrival time: 20:08
Estimated trip duration: 26 min


AI: Do you have any other requests?


User:Yes please, I would like to know the weather today in Gothenburg


AI: Okay! I understood that you are searching for the weather's forecast i